<a href="https://colab.research.google.com/github/Ochodus/DBLab_onehot/blob/main/one_hot_encoding(Variant_models).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [152]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [170]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn import svm
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.model_selection import train_test_split

In [171]:
data_subset = pd.read_excel('drive/MyDrive/DBLab_Onehot/subset_data.xlsx') #확장 데이터
data_original = pd.read_excel('drive/MyDrive/DBLab_Onehot/mental_health_medical_data.xlsx') #기존 데이터

In [172]:
data = pd.concat([data_subset, data_original]) #stratify 오류 방지를 위해 각 라벨 별로 최소 2개 이상의 데이터를 가지기 위한 concat

In [173]:
# 증상의 문자열 벗기기 함수
def listing(x):
    x = eval(x)
    return x

#모델 예측 정확도를 구하기 위한 간단한 함수
def getAccuracy(prediction, target):
  count = 0
  for x, y in zip(prediction, target):
    if x == y:
      count += 1  
  return count/len(prediction) * 100

data['symptoms'] = data['symptoms'].apply(listing)

In [174]:
#레이블 인코딩(타겟 문자열 불가)
from sklearn.preprocessing import LabelEncoder
items = data['disease']
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)

print('인코딩 변환값:',labels)

인코딩 변환값: [ 0  2  2 ... 11 58  4]


In [175]:
data

,Unnamed: 0,disease,symptoms,department
0,0.0,강박 장애,"[불안, 반복적인 동작, 의심]",정신건강의학과
1,1.0,경계성 인격장애,"[의심, 약물남용, 대인관계 어려움, 조절할 수 없는 충동감, 자살의 위험, 우울,...",정신건강의학과
2,2.0,경계성 인격장애,"[의심, 약물남용, 대인관계 어려움, 조절할 수 없는 충동감, 자살의 위험, 우울]",정신건강의학과
3,3.0,경계성 인격장애,"[약물남용, 대인관계 어려움, 조절할 수 없는 충동감, 자살의 위험, 우울]",정신건강의학과
4,4.0,경계성 인격장애,"[의심, 대인관계 어려움, 조절할 수 없는 충동감, 자살의 위험, 우울]",정신건강의학과
...,...,...,...,...
59,59.0,혈관성 치매,"[언어장애, 인지장애, 감각 이상, 마비]",정신건강의학과
60,60.0,히스테리성 인격장애,[],정신건강의학과
61,61.0,레트 증후군,"[상동적 행동, 인지장애, 운동장애, 정신 지체]",정신건강의학과
62,62.0,폐소공포증,"[불안, 좁은 공간에서의 불안감]",정신건강의학과


In [176]:
X_train = np.array([x for x in data['symptoms']])
Y_train = np.array(labels)

In [177]:
x_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, test_size=0.2, shuffle=True, stratify = Y_train, random_state=34) #0.8의 학습데이터, 0.2의 테스트데이터로 분리

# **모델링**

In [178]:
#파라미터 설정
vocab_size = 178
embedding_dim = 8 # 임베딩 차원
max_length = 17      # 증상 최대 개수 
padding_type='post'

In [179]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index

In [180]:
word_index

{'손떨림': 1,
 '가슴 두근거림': 2,
 '얼굴이 화끈거림': 3,
 '가슴 답답': 4,
 '온몸이 떨림': 5,
 '어지러움': 6,
 '두통': 7,
 '오심': 8,
 '구강건조': 9,
 '복부팽만감': 10,
 '구토': 11,
 '근육통': 12,
 '권태감': 13,
 '생리불순': 14,
 '가슴 쓰림': 15,
 '저림': 16,
 '불면증': 17,
 '환각': 18,
 '불안': 19,
 '환청': 20,
 '가슴 통증': 21,
 '기운없음': 22,
 '오한': 23,
 '발한': 24,
 '복시': 25,
 '서맥': 26,
 '숨막히는 느낌': 27,
 '고혈압': 28,
 '죽음에 대한 공포': 29,
 '저혈압': 30,
 '청색증': 31,
 '감정 변화': 32,
 '언어장애': 33,
 '초조함': 34,
 '섬망': 35,
 '혼돈': 36,
 '환시': 37,
 '수면장애': 38,
 '감각 이상': 39,
 '기억장애': 40,
 '피로감': 41,
 '실인증': 42,
 '목의 이물감': 43,
 '조급증': 44,
 '실행증': 45,
 '방향감각 상실': 46,
 '빈뇨': 47,
 '판단력장애': 48,
 '빈호흡': 49,
 '시력 감소': 50,
 '근육긴장': 51,
 '복부 불편감': 52,
 '낮은 자존감': 53,
 '의식 변화': 54,
 '대인관계 어려움': 55,
 '우울': 56,
 '망상': 57,
 '건망증': 58,
 '산만함': 59,
 '인지장애': 60,
 '수면 무호흡': 61,
 '인내심이 부족함': 62,
 '대화에 불쑥 끼어듬': 63,
 '복부 통증': 64,
 '수면 유지가 어려움': 65,
 '낮 시간대의 졸음': 66,
 '부정맥': 67,
 '수면 시작이 어려움': 68,
 '와해된 행동': 69,
 '수면 중 이상한 행동': 70,
 '호흡곤란': 71,
 '입수면기의 환각': 72,
 '수면 주기의 변화': 73,
 '흉부압박감': 74,
 '와해된 언어'

In [181]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(x_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(x_valid)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

In [182]:
test_padded

array([[ 5, 14,  1, ...,  0,  0,  0],
       [ 3, 12,  7, ...,  0,  0,  0],
       [ 5,  8, 14, ...,  0,  0,  0],
       ...,
       [ 8,  3, 13, ...,  0,  0,  0],
       [ 1,  4,  2, ...,  0,  0,  0],
       [ 8,  4,  2, ...,  0,  0,  0]], dtype=int32)

# ***NLP 모델***

In [183]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(64, activation='softmax')
])

In [184]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 17, 8)             1424      
                                                                 
 global_average_pooling1d_4   (None, 8)                0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_8 (Dense)             (None, 24)                216       
                                                                 
 dense_9 (Dense)             (None, 64)                1600      
                                                                 
Total params: 3,240
Trainable params: 3,240
Non-trainable params: 0
_________________________________________________________________
None


In [185]:
# fit model
num_epochs = 20
#history = model.fit(train_padded, y_train, 
                    #epochs=1000, verbose=2)
history = model.fit(train_padded, y_train, 
                     epochs=num_epochs, verbose=2, 
                     validation_split=0.2)

Epoch 1/20
1577/1577 - 3s - loss: 0.6236 - accuracy: 0.8887 - val_loss: 0.1849 - val_accuracy: 0.9493 - 3s/epoch - 2ms/step
Epoch 2/20
1577/1577 - 3s - loss: 0.1167 - accuracy: 0.9706 - val_loss: 0.0903 - val_accuracy: 0.9785 - 3s/epoch - 2ms/step
Epoch 3/20
1577/1577 - 3s - loss: 0.0669 - accuracy: 0.9838 - val_loss: 0.0622 - val_accuracy: 0.9860 - 3s/epoch - 2ms/step
Epoch 4/20
1577/1577 - 3s - loss: 0.0466 - accuracy: 0.9884 - val_loss: 0.0490 - val_accuracy: 0.9869 - 3s/epoch - 2ms/step
Epoch 5/20
1577/1577 - 5s - loss: 0.0360 - accuracy: 0.9897 - val_loss: 0.0395 - val_accuracy: 0.9905 - 5s/epoch - 3ms/step
Epoch 6/20
1577/1577 - 4s - loss: 0.0283 - accuracy: 0.9930 - val_loss: 0.0332 - val_accuracy: 0.9923 - 4s/epoch - 3ms/step
Epoch 7/20
1577/1577 - 3s - loss: 0.0231 - accuracy: 0.9939 - val_loss: 0.0282 - val_accuracy: 0.9935 - 3s/epoch - 2ms/step
Epoch 8/20
1577/1577 - 3s - loss: 0.0196 - accuracy: 0.9945 - val_loss: 0.0252 - val_accuracy: 0.9936 - 3s/epoch - 2ms/step
Epoch 9/

In [186]:
# 학습된 모델로 테스트 데이터 예측
y_prob = model.predict(test_padded, verbose=2) 
predicted = y_prob.argmax(axis=-1)

encoder.classes_[predicted]

493/493 - 1s - 542ms/epoch - 1ms/step


array(['신체형 장애', '신체형 장애', '신체형 장애', ..., '신체형 장애', '신체형 장애', '신체형 장애'],
      dtype=object)

In [187]:
getAccuracy(predicted, y_valid)

99.70821439898509

# ***SVM***


In [ ]:
#SupportVector 모델 학습
SVM = svm.SVC(C=1.0, kernel='poly', degree=3, gamma='auto', verbose=True)
SVM.fit(train_padded, y_train)

[LibSVM]

In [ ]:
predictions_SVM = SVM.predict(test_padded)
predictions_SVM

encoder.classes_[predictions_SVM]

# ***Naive Bayes***

In [188]:
#Naive bayes 모델 학습
Naive = naive_bayes.MultinomialNB()
Naive.fit(train_padded, y_train)

MultinomialNB()

In [189]:
predictions_NB = Naive.predict(test_padded)

encoder.classes_[predictions_NB]

array(['신체형 장애', '강박 장애', '신체형 장애', ..., '금단 현상', '신체형 장애', '신체형 장애'],
      dtype=object)

In [190]:
getAccuracy(predictions_NB, y_valid)

58.88360291785602

# ***Decision Tree***

In [191]:
#DecisionTree 모델 학습
DTC = tree.DecisionTreeClassifier()
DTC.fit(train_padded, y_train)

DecisionTreeClassifier()

In [192]:
predictions_DTC = DTC.predict(test_padded)

encoder.classes_[predictions_DTC]

array(['신체형 장애', '신체형 장애', '신체형 장애', ..., '신체형 장애', '신체형 장애', '신체형 장애'],
      dtype=object)

In [193]:
getAccuracy(predictions_DTC, y_valid)

99.91753885188709

---
# 여기서부터 추가 



더이상 입력할 증상이 없으면 종료
90%는 넘겨보자

안되면 어떤 증상을 물어보지?

현재 입력한 증상이 있는 질병중 가장 많이 출현하는 증상을 물어보자 예를 들면

"당신은 (해당 증상)을 가졌을 확률이 높습니다."

전제
1. 환자가 거짓된 증상을 입력하지 않는다
2. 오타가 나지 않는다 >> 뛰어쓰기같은건 replace(" ", "")로 처음부터 지우고 나머지는 다시 하라고 하자

In [ ]:
#증상 목록 출력
for i in word_index.keys():
  print(i)

#첫 증상 입력

##증상 입력받을 배열
pat_word_index = []

pat_symp = input("\n위 증상중 해당하는 증상을 하나씩 입력해주세요 (없으면 엔터) : ")
pat_word_index.append(pat_symp)

pat_word_array = np.array(pd.Series([pat_word_index]))

##데이터를 sequence로 변환해주고 padding 해줍니다.
pat_sequences = tokenizer.texts_to_sequences(pat_word_array.tolist())
pat_padded = pad_sequences(pat_sequences, padding=padding_type, maxlen=max_length)

##확률 계산
y_prob = model.predict(pat_padded, verbose=2)
predicted = y_prob.argmax(axis=-1)

#두번째 이상의 증상 입력
while(y_prob[0][predicted[0]] < 0.9 and pat_symp !=''):

  print("현재 확률은 ", y_prob[0][predicted[0]], "입니다")

  ##"당신은 (해당 증상)을 가졌을 확률이 높습니다."
  #sample_train = train.query('pat_word_index in train[\'symptoms\']').copy()
  sample_train = np.where(pd.Series(pat_word_index) in train['symptoms'])
  print(sample_train)

  pat_symp = input("\n증상을 추가로 입력해주세요 (없으면 엔터) : ")
  pat_word_index.append(pat_symp)

  pat_word_array = np.array(pd.Series([pat_word_index]))

  ##데이터를 sequence로 변환해주고 padding 해줍니다.
  pat_sequences = tokenizer.texts_to_sequences(pat_word_array.tolist())
  pat_padded = pad_sequences(pat_sequences, padding=padding_type, maxlen=max_length)

  ##확률 계산
  y_prob = model.predict(pat_padded, verbose=2)
  predicted = y_prob.argmax(axis=-1)

불안
우울
언어장애
대인관계 어려움
망상
감정 변화
수면장애
가슴 두근거림
불면증
환각
산만함
기억장애
어지러움
손떨림
초조함
인지장애
의심
조절할 수 없는 충동감
얼굴이 화끈거림
감각 이상
체중감소
구토
낮은 자존감
의식 변화
긴장
혼돈
충동 조절 어려움
정신 지체
약물남용
자살의 위험
발한
가슴 답답
오심
온몸이 떨림
저혈압
두통
환청
와해된 언어
피해망상
환시
와해된 행동
이상행동
식욕부진
피로감
무월경
복부 통증
폭식
권태감
복부팽만감
저림
비현실감
피부 긴장도 저하
이상한 소리를 냄
비정상적인 움직임
낮은 지능
행동장애
지남력 장애
섬망
대화에 불쑥 끼어듬
인내심이 부족함
건망증
몽유병
수면 무호흡
성격의변화
낮은 학업 성취
상동적 행동
반복적인 동작
숨막히는 느낌
가슴 통증
죽음에 대한 공포
오한
복시
고혈압
서맥
청색증
기운없음
수면 후에도 피로함
거칠어진 피부
빈호흡
빈뇨
복부 불편감
목의 이물감
근육긴장
설사
체중증가
생리불순
구강건조
근육통
가슴 쓰림
감각 과민
반사회적 인격장애
금단증상
말을 많이 함
높아진 자존심
사고의 비약
삶에 대한 흥미 상실
오후 우울감 완화
희망이 없는 느낌
걸핏하면 눈물지음
무욕증
정서적 둔마
무논리증
다리 통증
병원 쇼핑
질병에 대한 공포
우유부단함
에너지가 적음
비관주의
읽기
쓰기 능력 부족
반신마비
시야장애
말더듬
고정된 믿음
거짓 증상 호소
직무 태만
나쁜 품행
야만적 행위
사회성 부족
저체중
말초 신경장애
보행이상
안구진탕
신경과민
충동적 도벽
흥분
애착 상대와 분리 거부
왜곡된 신체상
자제력 상실 두려움
음식섭취 거부
피학증
가학증
소아기호증
노출증
비정상적 성적 기호
접촉도착증
관음증
발기부전
입수면기의 환각
수면 마비
수면 중 이상한 행동
수면 주기의 변화
수면 시작이 어려움
낮 시간대의 졸음
수면 유지가 어려움
수면 중 보행
깨어나면 기억하지 못함
실인증
방향감각 상실
시력 감소
실행증
판단력장애
조급증
읽기 자체의 문제
읽으려 하지 않음
눈의 피로
광과민성
눈의 충혈
변실금
이성의 옷을 착용함
강

TypeError: ignored

In [ ]:
print(type(train['symptoms']))
print(train['symptoms'])
print(type(pd.Series(pat_word_index)))
print(pd.Series(pat_word_index))

print(pd.Series(pat_word_index) in train['symptoms'])

<class 'pandas.core.series.Series'>
0        [불안, 반복적인 동작, 의심]
1            [불안, 반복적인 동작]
2                [반복적인 동작]
3                     [불안]
4                       []
               ...        
79827              [의식 변화]
79828          [흉부압박감, 불안]
79829                 [불안]
79830              [흉부압박감]
79831                   []
Name: symptoms, Length: 79832, dtype: object


NameError: ignored

In [ ]:
#결과 출력

##확률이 90%가 넘어서 멈춘 경우
if(y_prob[0][predicted[0]] >0.9):
  print("당신의 증상은 "+str(y_prob[0][predicted[0]])+"%의 확률로 "+encoder.classes_[predicted][0]+" 입니다.")

##더이상 입력을 안한 경우, 즉 입력을 완료한 경우
else:
  print("현재 입력해주신 증상으로는 "+str(y_prob[0][predicted[0]])+"%의 확률로 "+encoder.classes_[predicted][0]+" 입니다.")

현재 입력해주신 증상으로는 0.2940696%의 확률로 히스테리성 인격장애 입니다.


해야 할 내용

---

* 알코올성 치매, 히스테리성 인격장애는 증상이 없고, 증상이 없는 경우 확률은 30%대가 최대라 판별이 불가능한데 어떻게 해결할까 >> 일정 확률 이상의 질병들을 모두 출력할까?